In [76]:
import cv2
import numpy as np
import os


In [85]:
#importing the training data after uploading it to google collab
filenames = os.listdir('/content/drive/MyDrive/Colab Notebooks/training_images')

In [79]:

# Specify the input and output folder paths
input_folder = '/content/drive/MyDrive/Colab Notebooks/training_images'
output_folder = '/content/cropped_images'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

filenames1 = []

for i in range(len(filenames)):
    input_image_path = os.path.join(input_folder, f"{i}.png")
    output_image_path = os.path.join(output_folder, f"{i}.png")

    I = cv2.imread(input_image_path, 0)

    crop = I[0:100, 358:458]

    _, I_binary = cv2.threshold(crop, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    H = np.ones((3, 3), dtype=np.float32) / 9

    I_fil = I_binary

    SE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))

    # Perform morphological closing
    Iclosed = cv2.morphologyEx(I_fil, cv2.MORPH_CLOSE, SE)

    # Subtract the closed image from the filtered image
    Isub = cv2.subtract(Iclosed, I_fil)

    # Invert the subtracted image
    I_new = cv2.bitwise_not(Isub)

    # Subtract the inverted image from the filtered image
    I_clear = cv2.bitwise_not((cv2.subtract(I_new, I_fil)))
    I_clear_resized = cv2.resize(I_clear, (100, 100))  # Adjust the size as needed

    # Save the processed image
    cv2.imwrite(output_image_path, I_clear_resized)

    filenames1.append(output_image_path)

print("Images processed and saved to the output folder.")



Images processed and saved to the output folder.


In [81]:
import shutil
image_folder = "/content/cropped_images"
label_file = '/content/train_labels.txt'
output_folder = "classes"

# Create output folders if they don't exist
even_folder = os.path.join(output_folder, "even")
odd_folder = os.path.join(output_folder, "odd")
os.makedirs(even_folder, exist_ok=True)
os.makedirs(odd_folder, exist_ok=True)

# Read the labels from the file
with open(label_file, 'r') as file:
    labels = file.read().splitlines()

# Iterate over the labels and move the corresponding images
for i, label in enumerate(labels):
    image_name = f"{i}.png"
    source_path = os.path.join(image_folder, image_name)
    if label == "EVEN":
        destination_path = os.path.join(even_folder, image_name)
    elif label == "ODD":
        destination_path = os.path.join(odd_folder, image_name)
    else:
        continue  # Skip if the label is neither EVEN nor ODD
    shutil.copyfile(source_path, destination_path)

print("Images separated into 'odd' and 'even' folders.")

Images separated into 'odd' and 'even' folders.


In [82]:
#just checking the number of odd and even data
odd = os.listdir('/content/new/odd')
print(len(odd))

even = os.listdir('/content/new/even')
print(len(even))

1017
983


In [83]:
# Specify the paths for processed images and labels file
images_folder = '/content/cropped_images'
labels_file = '/content/train_labels.txt'

# Read labels from train_labels.txt file
with open(labels_file, 'r') as f:
    labels = f.read().splitlines()

data = []

# Iterate through processed images, flatten and combine with labels
for i, label in enumerate(labels):
    image_filename = f"{i}.png"
    image_path = os.path.join(images_folder, image_filename)

    cap_img = cv2.imread(image_path, 0)
    if cap_img is not None:
        image = np.array(cap_img).flatten()
        data.append([image, label])
    else:
        print(f"Error loading image: {image_path}")

print("Image features and labels combined.")

Image features and labels combined.


In [84]:
import pickle
from sklearn.svm import SVC

features = []
labels = []

for feature, label in data:
    features.append(feature)
    labels.append(label)

model = SVC(C=1, kernel='poly', gamma='auto')
model.fit(features, labels)

#saving the model and using in testing.ipynb
pick = open('model.sav', 'wb')
pickle.dump(model, pick)
pick.close()